In [1]:
# Project two part 1

# Scrape the content of CBC's 's Popular now section and save it as a CSV.

# We want: titles, article URL, byline, article type, image URL.

# Bonus, if you want to get fancy:

#     Make the CSV file auto-updating. Use this tutorial (https://www.youtube.com/watch?v=QNKxzkNpsko) (https://jonathansoma.com/everything/git/auto-updating-scaper-viz/) 


In [2]:
#Columns: title, subhead, article URL, whether it's premium or not, byline, article type, image URL.

In [23]:
import os
import random
import time

from playwright.async_api import async_playwright, expect

In [24]:
os.makedirs('data/', exist_ok=True)

In [25]:
async def open_browser(headless=False):
    """
    Starts the automated browser and opens a new window
    """
    # Start playwright
    playwright = await async_playwright().start()

    # Assign a random user agent (optional, shown below)
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"

    # Open firefox browser, can use chromium or webkit
    browser = await playwright.firefox.launch(headless=headless)

    # Create a new browser context (for setting user agent, etc.)
    context = await browser.new_context(user_agent=user_agent)

    # Create a new browser window (tab)
    page = await context.new_page()

    return browser, page

In [26]:
url = "https://www.cbc.ca/news"

In [27]:
async def open_browser(headless=False):
    playwright = await async_playwright().start()
    user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
    browser = await playwright.firefox.launch(headless=headless)
    context = await browser.new_context(user_agent=user_agent)
    page = await context.new_page()
    return browser, page

In [28]:
async def save_html_from_url(url, filename="cbc_homepage.html"):
    browser, page = await open_browser()
    await page.goto(url, timeout=60000)
    # Wait for the top stories section to load (vital for dynamic JS pages)
    await page.wait_for_selector('ol[data-cy="topStories"]', timeout=10000)
    # Optional: wait extra time if needed
    await page.wait_for_timeout(2000)

    # Get and save HTML content
    html = await page.content()
    with open(filename, "w", encoding="utf-8") as f:
        f.write(html)

    await browser.close()

In [ ]:
#save the html file for parsing
await save_html_from_url(url)

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [11]:
# Load the saved HTML
with open("cbc_homepage.html", "r", encoding="utf-8") as f:
    html = f.read()

doc = BeautifulSoup(html, "html.parser")

In [12]:
# #this told me the whole page isn't being loaded, so I need to switch to playwright to get the page, and then can return to beautiful soup
# with open("CBC_homepage.html", "w", encoding="utf-8") as f:
#     f.write(response.text)

# print("HTML saved to CBC_homepage.html")

In [14]:
# Find the container with the popular stories
popular_section = doc.select_one('ol[data-cy="topStories"]')

#find all the article in the trending section (there are 5) 
cards = popular_section.find_all("li", class_="card")
print(f"Found {len(cards)} top stories.")


Found 5 top stories.


In [15]:
# Loop through the eight elements in the list, pulling the information I want 
for article in cards:
    rank_tag = article.find("span", class_="rankIndicator")
    title_tag = article.find("h3", class_="headline")
    link_tag = article.find("a", class_="headlineLink")

    rank = rank_tag.get_text(strip=True) if rank_tag else 'No rank'
    title = title_tag.get_text(strip=True) if title_tag else 'No title'
    href = link_tag.get('href', '')
    full_url = url + href if href.startswith('/') else href

    print("Rank:", rank)
    print("Title:", title)
    print("URL:", full_url)
    print("---")

Rank: 1
Title: Father of Montreal girl who was found dead in N.Y. charged with 2nd-degree murder
URL: https://www.cbc.ca/news/news/canada/montreal/missing-canadian-girl-found-dead-new-york-1.7589787
---
Rank: 2
Title: Their neighbours changed the coastline and they want it fixed
URL: https://www.cbc.ca/news/news/canada/nova-scotia/wards-brook-beach-coastal-changes-1.7586621
---
Rank: 3
Title: Republican senator says Trump's 51st state rhetoric not 'constructive' during Ottawa visit
URL: https://www.cbc.ca/news/news/politics/trump-51st-state-comments-not-constructive-us-senator-says-1.7590032
---
Rank: 4
Title: 5 million above-ground pools recalled, including in Canada
URL: https://www.cbc.ca/news/news/canada/pool-recall-canada-1.7590213
---
Rank: 5
Title: The Cosby Show star Malcolm-Jamal Warner dead at 54
URL: https://www.cbc.ca/news/news/entertainment/malcolm-jamal-warner-obituary-1.7590147
---


In [16]:
rows = []  # List to store all article dicts

for article in cards:
    row = {}
    # Rank (required) 
    rank_tag = article.find("span", class_="rankIndicator")
    row['Rank'] = rank_tag.get_text(strip=True) if rank_tag else 'No rank'
    
    # Title (required)
    title_tag = article.find("h3", class_="headline")
    row['Article title'] = title_tag.get_text(strip=True) if title_tag else 'No title'

    # URL (required)
    link_tag = article.find("a", class_="headlineLink")
    href = link_tag.get('href', '')
    base_url = "https://www.theglobeandmail.com"
    row['url'] = base_url + href if href.startswith('/') else href

    rows.append(row)

print(f"Total articles scraped: {len(rows)}")


Total articles scraped: 5


In [17]:
df = pd.json_normalize(rows)

In [18]:
df.head(10)

,Rank,Article title,url
0,1,Father of Montreal girl who was found dead in ...,https://www.theglobeandmail.com/news/canada/mo...
1,2,Their neighbours changed the coastline and the...,https://www.theglobeandmail.com/news/canada/no...
2,3,Republican senator says Trump's 51st state rhe...,https://www.theglobeandmail.com/news/politics/...
3,4,"5 million above-ground pools recalled, includi...",https://www.theglobeandmail.com/news/canada/po...
4,5,The Cosby Show star Malcolm-Jamal Warner dead ...,https://www.theglobeandmail.com/news/entertain...


In [21]:
df["title_with_link"] = df.apply(lambda row: f'<a href="{row["url"]}">{row["Article title"]}</a>', axis=1)


In [22]:
df.to_csv("CBCScrape.csv", index=False)